# Web Scraping

Apartir destes dados vamos elaborar uma dashboard sobre Alguns filmes usando web scraping em python para colher as informações.

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
#lista de urls
urls = [
    'https://www.adorocinema.com/filmes/filme-269223/criticas/espectadores/',
    'https://www.adorocinema.com/filmes/filme-269223/criticas/espectadores/?page=2',
    'https://www.adorocinema.com/filmes/filme-269223/criticas/espectadores/?page=3',
    'https://www.adorocinema.com/filmes/filme-269223/criticas/espectadores/?page=4',
    'https://www.adorocinema.com/filmes/filme-269223/criticas/espectadores/?page=5',
    'https://www.adorocinema.com/filmes/filme-269223/criticas/espectadores/?page=6',
    'https://www.adorocinema.com/filmes/filme-269223/criticas/espectadores/?page=7'
]


In [4]:
# Extrator

def extrator(urls):
    todas_criticas = {
        "Nome": [],
        "Estrelas": [],
        "Data": [],
        "Descrição": []
    }


    for url in urls:
        # Fazer a requisição HTTP para obter o conteúdo da página
        response = requests.get(url)

        # Verificar se a requisição foi bem-sucedida
        if response.status_code == 200:
            print(f"Página acessada com sucesso: {url}")

            # Criar o objeto BeautifulSoup
            soup = BeautifulSoup(response.content, 'html.parser')

            #Encontra as Tags
            tag_pesq = soup.find_all('div', class_='review-card')

            for card in tag_pesq:
                # Extrair nomes dos críticos
                nome_element = card.find(class_="meta-title")
                todas_criticas["Nome"].append(nome_element.get_text(strip=True) if nome_element else None)

                # Extrair avaliações em estrelas
                estrelas_element = card.find(class_="stareval-note")
                todas_criticas["Estrelas"].append(estrelas_element.get_text(strip=True) if estrelas_element else None)

                # Extrair datas das críticas
                data_element = card.find(class_="review-card-meta-date light")
                todas_criticas["Data"].append(data_element.get_text(strip=True) if data_element else None)

                # Extrair descrições das críticas
                descricao_element = card.find(class_="content-txt review-card-content")
                todas_criticas["Descrição"].append(descricao_element.get_text(strip=True) if descricao_element else None)

        else:
            print(f"Falha ao acessar a página {url}. Status code: {response.status_code}")

    # Converter para DataFrame para facilitar análise
    df_criticas = pd.DataFrame(todas_criticas)

    return df_criticas

## Vamos executar o extrator

In [5]:
df_criticas = extrator(urls)

Página acessada com sucesso: https://www.adorocinema.com/filmes/filme-269223/criticas/espectadores/
Página acessada com sucesso: https://www.adorocinema.com/filmes/filme-269223/criticas/espectadores/?page=2
Página acessada com sucesso: https://www.adorocinema.com/filmes/filme-269223/criticas/espectadores/?page=3
Página acessada com sucesso: https://www.adorocinema.com/filmes/filme-269223/criticas/espectadores/?page=4
Página acessada com sucesso: https://www.adorocinema.com/filmes/filme-269223/criticas/espectadores/?page=5
Página acessada com sucesso: https://www.adorocinema.com/filmes/filme-269223/criticas/espectadores/?page=6
Página acessada com sucesso: https://www.adorocinema.com/filmes/filme-269223/criticas/espectadores/?page=7


In [6]:
df_criticas.to_csv('criticas_gladiador2.csv', index=False, encoding='utf-8')

print("Arquivo CSV criado com sucesso")
print(f"Total de críticas coletadas: {len(df_criticas)}")


Arquivo CSV criado com sucesso
Total de críticas coletadas: 66


# Analise de sentimento

A análise de sentimento é uma técnica que utiliza processamento de linguagem natural (NLP), aprendizado de máquina e estatísticas para identificar e categorizar emoções expressas em um texto. Essa análise é amplamente usada para entender melhor as percepções, opiniões e emoções de pessoas em relação a produtos, serviços, marcas, eventos, ou até mesmo tópicos específicos.

# Usando SpaCy

- Modelo de linguagem dedicado para português
- Melhor compreensão linguística
- Mais preciso na análise de sentimento
- Suporta tokenização e reconhecimento de entidades
- Treinado para o contexto brasileiro

In [10]:
#instale as dependencias
!pip install spacy textblob
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 68.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [11]:
#agora as bibliotecas
import pandas as pd
import spacy
from textblob import TextBlob

In [12]:
# carregando o modelo
nlp = spacy.load('pt_core_news_sm')

In [13]:
# Função de análise de sentimento
def analise_sentimentos(text):

    blob = TextBlob(str(text))
    sentimento_score = blob.sentiment.polarity

    # Classificação do sentimento
    if sentimento_score > 0.05:
        return 'Positivo', sentimento_score
    elif sentimento_score < -0.05:
        return 'Negativo', sentimento_score
    else:
        return 'Neutro', sentimento_score

In [14]:
# Carregar dados
df = pd.read_csv('criticas_gladiador2.csv', encoding='utf-8')

In [15]:
#Aplicando a a Função analise de sentimentos
df['Sentimento'], df['Pontuacao_Sentimento'] = zip(*df['Descrição'].apply(analise_sentimentos))

# Etapa de Validação dos dados

In [16]:
df.head()

,Nome,Estrelas,Data,Descrição,Sentimento,Pontuacao_Sentimento
0,Lino G.,"3,5",Enviada em 19 de novembro de 2024,Não vá com muita expectativa: o filme é simple...,Neutro,0.0500
1,Nelson J,"0,5",Enviada em 15 de novembro de 2024,O que poderia em alguma medida salvar é o extr...,Positivo,0.1875
2,Elisa C.,"5,0",Enviada em 20 de novembro de 2024,"Amei, um filme épico e com uma mensagem bem fo...",Neutro,0.0000
3,Enoque C.,"5,0",Enviada em 15 de novembro de 2024,"Não escutem os fãs apaixonados de Gladiador, o...",Neutro,0.0000
4,Emerson Galvão,"5,0",Enviada em 20 de novembro de 2024,"Não consigo enterder a crítica, a muito não se...",Neutro,0.0000


In [17]:
#validar informações
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Nome                  66 non-null     object 
 1   Estrelas              66 non-null     object 
 2   Data                  66 non-null     object 
 3   Descrição             65 non-null     object 
 4   Sentimento            66 non-null     object 
 5   Pontuacao_Sentimento  66 non-null     float64
dtypes: float64(1), object(5)
memory usage: 3.2+ KB


In [18]:
#checar dados nulos
df.isnull().sum()

,0
Nome,0
Estrelas,0
Data,0
Descrição,1
Sentimento,0
Pontuacao_Sentimento,0


In [19]:
#checar dados duplicados
df.duplicated().sum()

0

Engenharia dos dados

In [20]:
#Removendo as "","" da estrela
df['Estrelas'] = df['Estrelas'].str.replace(',', '.')

In [21]:
df['Data'] = df['Data'].str.replace('Enviada em ', '')

In [22]:
df.head()

,Nome,Estrelas,Data,Descrição,Sentimento,Pontuacao_Sentimento
0,Lino G.,3.5,19 de novembro de 2024,Não vá com muita expectativa: o filme é simple...,Neutro,0.0500
1,Nelson J,0.5,15 de novembro de 2024,O que poderia em alguma medida salvar é o extr...,Positivo,0.1875
2,Elisa C.,5.0,20 de novembro de 2024,"Amei, um filme épico e com uma mensagem bem fo...",Neutro,0.0000
3,Enoque C.,5.0,15 de novembro de 2024,"Não escutem os fãs apaixonados de Gladiador, o...",Neutro,0.0000
4,Emerson Galvão,5.0,20 de novembro de 2024,"Não consigo enterder a crítica, a muito não se...",Neutro,0.0000


In [23]:
#salvar em csv
df.to_csv('criticas_gladiador_sentimento.csv', index=False, encoding='utf-8')